In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        d = json.load(fopen)
        d = [d_['permalink'] for d_ in d]
        d = [d_ for d_ in d if 'https://www.sinchew.com.my' in d_]
        urls.extend(d)
urls = sorted(list(set(urls)))
len(urls), len(files)

100%|█████████████████████████████████| 283672/283672 [01:18<00:00, 3636.65it/s]


(1323408, 283672)

In [3]:
cookies = {
    '_cc_id': '69f8b801beef15345eb5accdea253af',
    '_ga_7DBYFGQB53': 'GS1.1.1689437590.1.0.1689437594.56.0.0',
    'panoramaId_expiry': '1690862886721',
    'panoramaId': '3d188d49baab2a52ec90d189081bbd9563ce47cda78c615eee8b2fb23aa6b3cf',
    'panoramaIdType': 'panoIndiv',
    '_gid': 'GA1.3.1978373061.1690476285',
    '_ga_ZC8YQ6MG6R': 'GS1.1.1690476297.1.1.1690476891.60.0.0',
    '_ga_NEB6M2N59L': 'GS1.1.1690558676.8.0.1690558676.60.0.0',
    '_ga_5CQPDC4JNY': 'GS1.1.1690558676.8.0.1690558676.60.0.0',
    '_ga_4V6TCB2L5G': 'GS1.1.1690622272.1.0.1690622275.57.0.0',
    '_clck': '13v5by8|2|fdp|0|1291',
    '_ga_266B2KSWGM': 'GS1.1.1690622391.4.0.1690622393.58.0.0',
    '_ga': 'GA1.1.1781742389.1689434320',
    '_clsk': '1kidr7y|1690622394526|2|0|s.clarity.ms/collect',
    'FCNEC': '%5B%5B%22AKsRol_wjtNiM1vKbgpxNleSQ9sdvYc4ew2zK1KUiwGiMwTF3JpRyDwiMUeLGOfxHCLVUaqQTYO3h5jrLOgWdXoL1j-DbtvoXAYIXdOH1pivAljR5ap3gkffTQQXRoGCL9FrasSCSh2GOLbokNRphDvz1y_lzXfQQQ%3D%3D%22%5D%2Cnull%2C%5B%5D%5D',
    'cto_bundle': 'HIzyXl9iJTJCUyUyQkslMkZvaGRoNTElMkZlSTMwVm9UbXhmVHlQdU54WUxhTCUyQnBSTTNYYTNXeTk5bVNPbGpPTkdPQ3FJMTBlUTZ3ZW9TYm5TVEY3RkhCM2M3MlR1MDJpWEFEY0Y5UWhHS0dIJTJGMzRrQnFrYWFtY2VLQTlsY0h6VFJYM1l1UmZTY01qVVFwNCUyRkkzUU9zOGhWQUVFVHliMW9EdyUzRCUzRA',
    '_ga_74B2VR88L4': 'GS1.1.1690622272.9.1.1690622397.53.0.0',
    '_ga_1KX2CHDVKL': 'GS1.1.1690622393.1.0.1690622397.56.0.0',
    'ivs_video_2732927_lastPlayed': '34198,156760',
}

headers = {
    'authority': 'www.sinchew.com.my',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '_cc_id=69f8b801beef15345eb5accdea253af; _ga_7DBYFGQB53=GS1.1.1689437590.1.0.1689437594.56.0.0; panoramaId_expiry=1690862886721; panoramaId=3d188d49baab2a52ec90d189081bbd9563ce47cda78c615eee8b2fb23aa6b3cf; panoramaIdType=panoIndiv; _gid=GA1.3.1978373061.1690476285; _ga_ZC8YQ6MG6R=GS1.1.1690476297.1.1.1690476891.60.0.0; _ga_NEB6M2N59L=GS1.1.1690558676.8.0.1690558676.60.0.0; _ga_5CQPDC4JNY=GS1.1.1690558676.8.0.1690558676.60.0.0; _ga_4V6TCB2L5G=GS1.1.1690622272.1.0.1690622275.57.0.0; _clck=13v5by8|2|fdp|0|1291; _ga_266B2KSWGM=GS1.1.1690622391.4.0.1690622393.58.0.0; _ga=GA1.1.1781742389.1689434320; _clsk=1kidr7y|1690622394526|2|0|s.clarity.ms/collect; FCNEC=%5B%5B%22AKsRol_wjtNiM1vKbgpxNleSQ9sdvYc4ew2zK1KUiwGiMwTF3JpRyDwiMUeLGOfxHCLVUaqQTYO3h5jrLOgWdXoL1j-DbtvoXAYIXdOH1pivAljR5ap3gkffTQQXRoGCL9FrasSCSh2GOLbokNRphDvz1y_lzXfQQQ%3D%3D%22%5D%2Cnull%2C%5B%5D%5D; cto_bundle=HIzyXl9iJTJCUyUyQkslMkZvaGRoNTElMkZlSTMwVm9UbXhmVHlQdU54WUxhTCUyQnBSTTNYYTNXeTk5bVNPbGpPTkdPQ3FJMTBlUTZ3ZW9TYm5TVEY3RkhCM2M3MlR1MDJpWEFEY0Y5UWhHS0dIJTJGMzRrQnFrYWFtY2VLQTlsY0h6VFJYM1l1UmZTY01qVVFwNCUyRkkzUU9zOGhWQUVFVHliMW9EdyUzRCUzRA; _ga_74B2VR88L4=GS1.1.1690622272.9.1.1690622397.53.0.0; _ga_1KX2CHDVKL=GS1.1.1690622393.1.0.1690622397.56.0.0; ivs_video_2732927_lastPlayed=34198,156760',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [4]:
r = requests.get(urls[0], cookies = cookies, headers = headers)
soup = BeautifulSoup(r.content, "lxml")

In [5]:
soup.find('div', {'itemprop':"articleBody"}).get_text(separator="\n")

'\n\n\n\n导演：\n赵东五Dong-oh Jo\n\n\n演员：\n\n许俊豪Jun-ho Heo饰Ban-chu\n\n郑雨盛Woo-sung Jung饰Yi Gwak\n\n\n\n\n金光一Kwang-il Kim\n\n金泰熙Tae-hee Kim饰So-hwa/Yon-hwa\n\n朴正学Jeong-hak Park\n\n朴尚旭Sang-wook Park\n\n刘河俊Ha-jun Yu\n\n\n类型：\n动作、幻想、冒险、剧情、爱情\n\n\n语言：\n韩语\n\n\n故事简介：\n极度混乱的统一新罗末期，具有能看到鬼神能力的李阔（郑雨盛饰），因为恶鬼夺去了未成婚的妻子莲花（金泰熙饰），伤心之下投身到了王室的一只除魔队效力。\n\n\n由于李阔的超能力，他成为所在部队的第一武士，可是队友们却在恶鬼的迷惑下图谋叛变，但最后行动失败，所有人都被处以了极刑，先是失去比生命更重要的恋人，再和亲如兄弟般的战友生离死别。\n\n\n怒发冲冠的李阔发誓与魔界誓不两立，成了一名真正的除魔士。然而有一天，他却意外的通过一个神殿来到了只有死去的人才能到达的奇妙世界——中天。\n\n\n中天，是存在于天上和地下的第三世界，为49天后准备转生的灵魂们而存在的世界。李阔来在这里意外的遇到了极像莲花的女子。可是这个女子却一口咬定自己是天人小花，一点也没有认出痛苦万分，苦苦哀叹的李阔，不巧的是那些引起叛乱的恶鬼们也在这里，并对他展开了追杀。然而李阔虽然深知自己在这个异度空间中随时会遇到危险，但是为了再次相逢的莲花，他决定再也不会离开她。\n\n\n为了莲花可以不顾性命的李阔又一次意外的见到了不久前冤死的战友们，可是这时，战友们已经变成了真正的恶鬼，而李阔又执意要带莲花回在现世，面对着认不出自己的莲花，李阔尚可以一如既往的保护和爱护，可是面对着那些曾对犹如手足，并把自己从刑场里保全下来的战友们，手中的剑一时间有如千斤重…… \n\n'

In [6]:
def crawl(url, filename):
    if os.path.exists(filename):
        return [True]
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)

    soup = BeautifulSoup(r.content, "lxml")
    
    try:
        body = soup.find('div', {'itemprop':"articleBody"}).get_text(separator="\n")
    except:
        body = None
    
    data = {
        'url': url,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [7]:
# !rm -rf page
!mkdir page

mkdir: cannot create directory ‘page’: File exists


In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 20

for i in tqdm(range(0, len(urls), max_worker)):
    b = urls[i: i + max_worker]
    urls_ = [(b[k], os.path.join('page', f'{i + k}.json')) for k in range(len(b))]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████| 66171/66171 [7:31:03<00:00,  2.45it/s]
